In [74]:
import pandas as pd
import numpy as np


In [75]:
air_quality_fp = 'AQI and Lat Long of Countries.csv'
CountryAndCodeFP = 'API_NY.GDP.PCAP.CD_DS2_en_csv_v2_5454904.CSV'
country_gdp_fp = 'CountryGDP.csv'
country_continent_fp = 'countryContinent.csv'
cleaned_path = 'air quality\\datasets\\'

In [76]:
##view air quality datafram and perform some cleanups
AirQuality1 = pd.read_csv (air_quality_fp)
AirQuality1.sort_values((['Country', 'City']), ascending=[True, True], inplace= True, ignore_index=True)
AirQuality1.dropna()
#302 rows without countries

#check and remove records without country, city, or lat/long values
AirQuality1 = AirQuality1[(AirQuality1['lat'].notnull()) & (AirQuality1['lng'].notnull()) 
                  & (AirQuality1['Country'].notnull()) & (AirQuality1['City'].notnull()) ] 

In [77]:
##make dimcountry table
AirQuality1country = pd.DataFrame(AirQuality1['Country'].unique(), columns= ['Country'])
AirQuality1country.reset_index(drop=True, inplace= True)
AirQuality1country.insert(0,'CountryID', AirQuality1country.index + 1)
AirQuality1country

##make dimcity table
AirQuality1city = pd.DataFrame(AirQuality1['City'].drop_duplicates())
AirQuality1city.reset_index(drop=True, inplace= True)
AirQuality1city.insert(0,'CityID',AirQuality1city.index + 1)


##make dimbridge table
AirQuality1CountryCityBridge = pd.DataFrame(AirQuality1[['Country', 'City', 'lat','lng']])
AirQuality1CountryCityBridge.drop_duplicates(inplace= True)
AirQuality1CountryCityBridge = AirQuality1CountryCityBridge.merge(AirQuality1country, on='Country', how='left' ).merge(AirQuality1city, on='City', how='left')
AirQuality1CountryCityBridge.insert(0,'CountryCityID', AirQuality1CountryCityBridge.index + 1)
AirQuality1CountryCityBridge = AirQuality1CountryCityBridge[['CountryCityID', 'CountryID', 'CityID', 'lat', 'lng']]



In [78]:
CountryContinent = pd.read_csv(country_continent_fp, encoding = "ISO-8859-1")
CountryContinent = CountryContinent[['country', 'continent', 'sub_region']]
m1 = CountryContinent.merge(AirQuality1country, how='left', left_on='country', right_on= 'Country')
m2 = AirQuality1country.merge(CountryContinent, how='left', left_on='Country', right_on= 'country')

##countries in AirQuality1country that do not match with CountryContinent 
CountriesNotMatched = m2[m2['country'].isnull()]
CountriesNotMatched


,CountryID,Country,country,continent,sub_region
41,42,Czechia,NaN,NaN,NaN
43,44,Democratic Republic of the Congo,NaN,NaN,NaN
81,82,Kingdom of Eswatini,NaN,NaN,NaN
126,127,Republic of Korea,NaN,NaN,NaN
127,128,Republic of Moldova,NaN,NaN,NaN
128,129,Republic of North Macedonia,NaN,NaN,NaN
148,149,State of Palestine,NaN,NaN,NaN
165,166,United Republic of Tanzania,NaN,NaN,NaN


##### The countries below in AirQuality1country that did not have a match need <br> to be renamed to match official UN list 
Chechia : Czech Republic
Democratic Republic of the Congo : Congo (Democratic Republic of the)
Kingdom of Eswatini : Swaziland
Republic of Korea : Korea (Republic of)
Republic of Moldova	: Moldova (Republic of)
Republic of North Macedonia	: Macedonia (the former Yugoslav Republic of)
State of Palestine	: State of Palestine
United Republic of Tanzania	:	Tanzania, United Republic of


In [79]:
CountryRemap = {
'Chechia' : 'Czech Republic',
'Democratic Republic of the Congo ': 'Congo (Democratic Republic of the)',
'Kingdom of Eswatini' : 'Swaziland',
'Republic of Korea' : 'Korea (Republic of)',
'Republic of Moldova	': 'Moldova (Republic of)',
'Republic of North Macedonia'	: 'Macedonia (the former Yugoslav Republic of)',
'State of Palestine': 'State of Palestine',
'United Republic of Tanzania':	'Tanzania, United Republic of'
}

AirQuality1country = AirQuality1country.replace(to_replace={'Country': CountryRemap})

##spot check that relabel worked
AirQuality1country.loc[AirQuality1country['Country'].str.lower().str.contains('Korea|Swaziland|Palestine|Tanzania'.lower())]


,CountryID,Country
81,82,Swaziland
126,127,Korea (Republic of)
148,149,State of Palestine
165,166,"Tanzania, United Republic of"


In [80]:
##relabel AirQuality1 countries
##seems to map correctly. can assign to airquality1
AirQuality1.replace(to_replace={'Country': CountryRemap}).loc[AirQuality1['Country'].str.lower().str.contains('Korea|Swaziland|Palestine|Tanzania'.lower())]
AirQuality1 = AirQuality1.replace(to_replace={'Country': CountryRemap})


In [81]:
CountryContinent

,country,continent,sub_region
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia
...,...,...,...
244,Wallis and Futuna,Oceania,Polynesia
245,Western Sahara,Africa,Northern Africa
246,Yemen,Asia,Western Asia
247,Zambia,Africa,Eastern Africa


In [82]:
m1 = AirQuality1country.merge(AirQuality1, how= 'left', left_on= 'Country', right_on= 'Country').merge(AirQuality1city, how= 'left', left_on= 'City', right_on= 'City')
m1.columns

Index(['CountryID', 'Country', 'City', 'AQI Value', 'AQI Category',
       'CO AQI Value', 'CO AQI Category', 'Ozone AQI Value',
       'Ozone AQI Category', 'NO2 AQI Value', 'NO2 AQI Category',
       'PM2.5 AQI Value', 'PM2.5 AQI Category', 'lat', 'lng', 'CityID'],
      dtype='object')

In [83]:
AirQuality1 = m1[['CountryID', 'CityID', 'AQI Value', 'AQI Category',
       'CO AQI Value', 'CO AQI Category', 'Ozone AQI Value',
       'Ozone AQI Category', 'NO2 AQI Value', 'NO2 AQI Category',
       'PM2.5 AQI Value', 'PM2.5 AQI Category', 'lat', 'lng']]


#### adding further fields to countries

In order to draw further analysis on airquality, I want visualize in power BI what the relationship Between AQI and gdp and population is <br>
I will prep for this by appending the GDP csv to the AirQuality1country dataframe which will become the countries dimension table.<br>
CountryGDP.CSV was taken from world bank data


In [84]:
countryGDP = pd.read_csv(country_gdp_fp)
m1 = countryGDP.merge( AirQuality1country, how= 'left', left_on= 'Country', right_on= 'Country')
m1[m1['CountryID'].isnull()]['Country'].to_list()
##Below are the countries in CountryGDP that are either spelled differently or do not map to anything in AirQuality1country.
## map countries with different spelling to countris in airquality table

['United States',
 'United Kingdom',
 'Russia',
 'South Korea',
 'Iran',
 'Hong Kong',
 'Vietnam',
 'Czech Republic (Czechia)',
 'Tanzania',
 'Macao',
 'DR Congo',
 'Bolivia',
 'Laos',
 'Bahamas',
 'Brunei',
 'North Macedonia',
 'Moldova',
 'Guam',
 'Fiji',
 'Eswatini',
 'Timor-Leste',
 'San Marino',
 'Northern Mariana Islands',
 'Antigua and Barbuda',
 'Grenada',
 'Saint Kitts & Nevis',
 'Vanuatu',
 'Samoa',
 'St. Vincent & Grenadines',
 'American Samoa',
 'Dominica',
 'Tonga',
 'Sao Tome & Principe',
 'Micronesia',
 'Marshall Islands',
 'Kiribati',
 'Tuvalu']

In [85]:
gdpCountriesRemap = {'United States': 'United States of America',
 'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
 'Russia': 'Russian Federation',
 'South Korea':  'Korea (Republic of)',
 'Iran': 'Iran (Islamic Republic of)',
 'Vietnam': 'Viet Nam',
 'Czech Republic (Czechia)':  'Czechia',
 'Tanzania':  'Tanzania, United Republic of',
 'DR Congo': 'Democratic Republic of the Congo',
 'Bolivia': 'Bolivia (Plurinational State of)',
 'Laos': "Lao People's Democratic Republic",
 'North Macedonia': 'Macedonia (the former Yugoslav Republic of)',
 'Moldova': 'Republic of Moldova',
 'Eswatini': 'Swaziland',
 'Saint Kitts & Nevis': 'Saint Kitts and Nevis'
 }

In [86]:
countryGDP = countryGDP.replace(to_replace={'Country':gdpCountriesRemap})
m1 = AirQuality1country.merge( countryGDP, how= 'left', left_on= 'Country', right_on= 'Country')
AirQuality1country = m1[['CountryID', 'Country', 'GDP nominal', 'GDP growth', 'Population ', 'GDP per capita', 'Share of World GDP']]
##AirQuality1country = AirQuality1country.rename(columns= { 'GDP nominal_x':"GDP nominal", 'GDP growth_x':'GDP growth', 'Population _x':'Population ', 'GDP per capita_x':'GDP per capita', 'Share of World GDP_x':'Share of World GDP'})
AirQuality1country

,CountryID,Country,GDP nominal,GDP growth,Population,GDP per capita,Share of World GDP
0,1,Afghanistan,"$19,543,976,895",2.67%,"36,296,113",$538,0.02%
1,2,Albania,"$13,038,538,300",3.84%,"2,884,169","$4,521",0.02%
2,3,Algeria,"$167,555,280,113",1.60%,"41,389,189","$4,048",0.21%
3,4,Andorra,"$3,012,914,131",1.87%,"77,001","$39,128",0.00%
4,5,Angola,"$122,123,822,334",-0.15%,"29,816,766","$4,096",0.15%
...,...,...,...,...,...,...,...
169,170,Venezuela (Bolivarian Republic of),NaN,NaN,NaN,NaN,NaN
170,171,Viet Nam,"$223,779,865,815",6.81%,"94,600,648","$2,366",0.28%
171,172,Yemen,"$31,267,675,216",-5.94%,"27,834,819","$1,123",0.04%
172,173,Zambia,"$25,868,142,073",3.40%,"16,853,599","$1,535",0.03%


In [103]:
AirQuality1country = m1[['CountryID', 'Country', 'continent',
       'sub_region','GDP nominal', 'GDP growth', 'Population ',
       'GDP per capita', 'Share of World GDP']]

Dataframes have now been cleaned and appended with other sources to be exported as CSV files. <br>
CSV files will be loaded into sql server tables with appropriate relationships then imported into Power BI

In [104]:

AirQuality1.to_csv(path_or_buf= cleaned_path + 'fact_air_quality.csv', index=False)
AirQuality1city.to_csv(path_or_buf= cleaned_path + 'dim_city.csv', index=False)
AirQuality1country.to_csv(path_or_buf= cleaned_path + 'dim_country.csv', index=False)
AirQuality1CountryCityBridge.to_csv(path_or_buf= cleaned_path + 'DimCountryCityBridge.csv', index=False)
